## How long does it take to process a permanent worker visa? ##

In [1]:
import pandas as pd
import glob
import time
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
init_notebook_mode(connected=True)
from plotly.graph_objs import *
#import plotly 
#import plotly.plotly as py
#import plotly.graph_objs as go
#import plotly.tools as tls
import us
import os
from iso3166 import countries

#plotly.tools.set_credentials_file(username='sneharavi15', api_key='w4b8nbp3q7')


perm_data = pd.read_pickle('perm')


In [2]:
perm_data_15 = perm_data[pd.DatetimeIndex(perm_data['DECISION_DATE']).year == 2015]
print(str(len(perm_data_15)) + ' Total Applications in 2015')

print(str(len(perm_data_15.dropna(how='any'))) + ' remaining applications after nulls dropped')
perm_data_15 = perm_data_15.dropna(how='any')

print("We don't want the cases where they withdrew the application because they were not processed and it doesn't make sense to take them into account.")
perm_data_15 = perm_data_15[perm_data_15['CASE_STATUS'].str.startswith('W') == False]
print(str(len(perm_data_15)) + ' Total Applications Processesd')
print(str(len(perm_data_15[perm_data_15['CASE_STATUS'].str.startswith("C")])) + " applications were accepted and " + str(len(perm_data_15[perm_data_15['CASE_STATUS'].str.startswith("D")]))+ " applications were denied.")
td = (perm_data_15['DECISION_DATE'] - perm_data_15['CASE_RECEIVED_DATE'])
perm_data_15["TIME_TAKEN"] = (td / np.timedelta64(1, 'D')).astype(int)
perm_data_15.head()

98453 Total Applications in 2015
85556 remaining applications after nulls dropped
We don't want the cases where they withdrew the application because they were not processed and it doesn't make sense to take them into account.
81740 Total Applications Processesd
77253 applications were accepted and 4487 applications were denied.


CASE_NUMBER DECISION_DATE        CASE_STATUS CASE_RECEIVED_DATE  \
15 0  A-14220-96665    2015-02-03  Certified-Expired         2014-09-10   
   1  A-14220-96720    2015-01-12  Certified-Expired         2014-08-08   
   4  A-14202-90786    2015-01-07  Certified-Expired         2014-08-25   
   6  A-14202-90920    2015-01-26  Certified-Expired         2014-07-25   
   7  A-14220-96712    2015-01-14  Certified-Expired         2014-08-11   

                            EMPLOYER_NAME                  PW_JOB_TITLE_9089  \
15 0               UNION PACIFIC RAILROAD  Software Developers, Applications   
   1                   VST CONSULTING INC  Software Developers, Applications   
   4           NET ESOLUTIONS CORPORATION  Software Developers, Applications   
   6  BURGER REHABILITATION SYSTEMS, INC.             Occupational Therapist   
   7                  SHUBH SOLUTIONS LLC  Software Developers, Applications   

     PW_LEVEL_9089 WAGE_OFFER_FROM_9089 WAGE_OFFER_UNIT_OF_PAY_9089  \
15 0     Level III            93,000.00                        Year   
   1     Level III            90,459.00                        Year   
   4      Level II            94,016.00                        Year   
   6      Level II           104,000.00                        Year   
   7      Level IV           111,717.00                        Year   

     JOB_INFO_WORK_CITY JOB_INFO_WORK_STATE COUNTRY_OF_CITIZENSHIP  \
15 0              Omaha            NEBRASKA                  INDIA   
   1             Iselin          NEW JERSEY                  INDIA   
   4             McLean            VIRGINIA                  INDIA   
   6        Porterville          CALIFORNIA            PHILIPPINES   
   7          San Diego          CALIFORNIA                  INDIA   

     CLASS_OF_ADMISSION  TIME_TAKEN  
15 0               H-1B         146  
   1               H-1B         157  
   4               H-1B         135  
   6               H-1B         185  
   7               H-1B         156

### Where are people applying from? ###

In [3]:
gbcountries = perm_data_15.groupby(['COUNTRY_OF_CITIZENSHIP']).size().reset_index(name='count')
pd.set_option('display.max_rows', len(gbcountries))
gbcountries['COUNTRY_OF_CITIZENSHIP_ABBR'] = ""

gbcountries["MEDIAN_TIME_TAKEN"] = perm_data_15.groupby('COUNTRY_OF_CITIZENSHIP')['TIME_TAKEN'].median().reset_index()['TIME_TAKEN']
gbcountries["MEAN_TIME_TAKEN"] = perm_data_15.groupby('COUNTRY_OF_CITIZENSHIP')['TIME_TAKEN'].mean().reset_index()['TIME_TAKEN']
gbcountries["STD_TIME_TAKEN"] = perm_data_15.groupby('COUNTRY_OF_CITIZENSHIP')['TIME_TAKEN'].std().reset_index()['TIME_TAKEN']
gbcountries['STD_TIME_TAKEN'] = gbcountries['STD_TIME_TAKEN'].fillna(0)

for i, c in enumerate(gbcountries['COUNTRY_OF_CITIZENSHIP']):
    if c == 'IVORY COAST':
        gbcountries = gbcountries[gbcountries['COUNTRY_OF_CITIZENSHIP'].str.contains('IVORY') == False]
    elif c == 'KOSOVO':
        gbcountries = gbcountries[gbcountries['COUNTRY_OF_CITIZENSHIP'].str.contains('KOSOVO') == False]
    elif c == 'PALESTINIAN TERRITORIES':
        gbcountries = gbcountries[gbcountries['COUNTRY_OF_CITIZENSHIP'].str.contains('PALESTINIAN') == False]
    elif c == 'DEMOCRATIC REPUBLIC OF CONGO':
        gbcountries = gbcountries[gbcountries['COUNTRY_OF_CITIZENSHIP'].str.contains('DEMOCRATIC REPUBLIC OF CONGO') == False]
    elif c == 'YUGOSLAVIA':
        gbcountries = gbcountries[gbcountries['COUNTRY_OF_CITIZENSHIP'].str.contains('YUGOSLAVIA') == False]

gbcountries.index = range(len(gbcountries))
    
for i, c in enumerate(gbcountries['COUNTRY_OF_CITIZENSHIP']):    
    if c == "BOLIVIA":
        gbcountries.set_value(i, "COUNTRY_OF_CITIZENSHIP", "BOL")
    elif c == 'BRITISH VIRGIN ISLANDS':
        gbcountries.set_value(i, "COUNTRY_OF_CITIZENSHIP", "VG")
    elif c == 'BURMA (MYANMAR)':
        gbcountries.set_value(i, "COUNTRY_OF_CITIZENSHIP", "MYANMAR")
    elif c == "COTE d'IVOIRE":
        gbcountries.set_value(i, "COUNTRY_OF_CITIZENSHIP", "CI")
        gbcountries.set_value(i, "count", 7)
    elif c == 'REPUBLIC OF CONGO':
        gbcountries.set_value(i, "COUNTRY_OF_CITIZENSHIP", "CONGO")
        gbcountries.set_value(i, "count", 5)
    elif c == 'IRAN':
        gbcountries.set_value(i, "COUNTRY_OF_CITIZENSHIP", "ir")
    elif c == 'MACAU':
        gbcountries.set_value(i, "COUNTRY_OF_CITIZENSHIP", "MACAO")
    elif c == 'MACEDONIA':
        gbcountries.set_value(i, "COUNTRY_OF_CITIZENSHIP", 'MK')
    elif c == 'MOLDOVA':
        gbcountries.set_value(i, "COUNTRY_OF_CITIZENSHIP", 'MDA')
    elif c == 'PALESTINE':
        gbcountries.set_value(i, "count", 23)
    elif c == 'RUSSIA':
        gbcountries.set_value(i, "COUNTRY_OF_CITIZENSHIP", 'RUS')
    elif c == 'SERBIA AND MONTENEGRO':
        gbcountries.set_value(i, "COUNTRY_OF_CITIZENSHIP", 'SERBIA')
    elif c == 'SOUTH KOREA':
        gbcountries.set_value(i, "COUNTRY_OF_CITIZENSHIP", 'KOR')
    elif c == 'ST KITTS AND NEVIS':
        gbcountries.set_value(i, "COUNTRY_OF_CITIZENSHIP", 'KN')
    elif c == 'ST LUCIA':
        gbcountries.set_value(i, "COUNTRY_OF_CITIZENSHIP", 'LC')
    elif c == 'SYRIA':
        gbcountries.set_value(i, "COUNTRY_OF_CITIZENSHIP", 'SY')
    elif c == 'TANZANIA':
        gbcountries.set_value(i, "COUNTRY_OF_CITIZENSHIP", 'TZ')
    elif c == 'UNITED STATES OF AMERICA':
        gbcountries.set_value(i, "COUNTRY_OF_CITIZENSHIP", 'UNITED STATES')
    elif c == 'VENEZUELA':
        gbcountries.set_value(i, "COUNTRY_OF_CITIZENSHIP", 'VEN')
    elif c == 'VIETNAM':
        gbcountries.set_value(i, "COUNTRY_OF_CITIZENSHIP", 'VIET NAM')
        
for i, c in enumerate(gbcountries['COUNTRY_OF_CITIZENSHIP']):
    gbcountries['COUNTRY_OF_CITIZENSHIP_ABBR'][i] = countries.get(c).alpha3
#gbcountries

/Users/sneharavi/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:69: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [4]:
scl = map(str,(np.array(sns.cubehelix_palette(15, start=0, rot=0, dark=0.2, light=0.95))*255).astype(int))
scl = [w.replace('[ ','rgb(') for w in scl]
scl = [w.replace('[','rgb(') for w in scl]
scl= [w.replace(']',')') for w in scl]
scl= [w.replace('  ',' ') for w in scl]
scl= [w.replace(' ',',') for w in scl]
fr = np.linspace(0.0, 1.0, 15).tolist()

clrscl = [list(a) for a in zip(fr, scl)]

gbcountries['text'] = gbcountries['COUNTRY_OF_CITIZENSHIP'] + '<br>' +\
    'Number of applications '+ gbcountries['count'].astype(str)

data = [ dict(type='choropleth',colorscale = clrscl,autocolorscale = False,locations = gbcountries['COUNTRY_OF_CITIZENSHIP_ABBR'],
        z = gbcountries['count'],
        text = gbcountries['text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            )
        ),
        colorbar = dict(
            title = "Number of applications"
        )
    ) ]

layout = dict(title = 'Number of Applications by Country<br>(Hover for breakdown)',geo = dict(showframe = False,showcoastlines = False,projection = dict(type = 'Mercator')))

#layout = dict(title = 'Number of Applications by Country<br>(Hover for breakdown)',geo = dict(scope='usa',projection=dict( type='albers usa' ),showlakes = True,lakecolor = 'rgb(255, 255, 255)'))
fig = dict(data=data, layout=layout)
iplot(fig)


In [5]:
countperc = (gbcountries['count']/gbcountries['count'].sum())*100
a =  countperc.sort_values(ascending = False)
b = np.array(countperc).argsort()[::-1][:len(a[a > 0.9])]

topcount = []
topperc = []
for val in a[b]:
    topperc.append(val)
topperc.append(18)
for c in gbcountries.loc[b]['COUNTRY_OF_CITIZENSHIP']:
    if c == "KOR":
        topcount.append("SOUTH KOREA")
    else:
        topcount.append(c)
topcount.append('OTHERS')

fig = {
    'data': [{'labels': topcount,
              'values': topperc,
              'type': 'pie',
            'hoverinfo':'none',
            'textinfo':'label+percent'}],
    'layout': {'title': 'Percentage of Applications by Prominent Countries'}
     }

iplot(fig)

### Does more people applying from certain aountries mean that it takes more time for applications to be processed in those countries? ###

In [6]:
scl = map(str,(np.array(sns.cubehelix_palette(6, start=0, rot=1, dark=0.2, light=0.95))*255).astype(int))
scl = [w.replace('[ ','rgb(') for w in scl]
scl = [w.replace('[','rgb(') for w in scl]
scl= [w.replace(']',')') for w in scl]
scl= [w.replace('  ',' ') for w in scl]
scl= [w.replace(' ',',') for w in scl]
fr = np.linspace(0.0, 1.0, 6).tolist()

clrscl = [list(a) for a in zip(fr, scl)]

gbcountries['text'] = gbcountries['COUNTRY_OF_CITIZENSHIP'] + '<br>' +\
    'Mean time '+ gbcountries['MEAN_TIME_TAKEN'].astype(int).astype(str) + '<br>' +\
     'Std '+ gbcountries['STD_TIME_TAKEN'].astype(int).astype(str) + '<br>' +\
      'No apps ' + gbcountries['count'].astype(str)

data = [ dict(type='choropleth',colorscale = clrscl,autocolorscale = False,locations = gbcountries['COUNTRY_OF_CITIZENSHIP_ABBR'],
        z = gbcountries['MEAN_TIME_TAKEN'],
        text = gbcountries['text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            )
        ),
        colorbar = dict(
            title = "Mean time taken"
        )
    ) ]

layout = dict(title = 'Mean time taken by Country<br>(Hover for breakdown)',geo = dict(showframe = False,showcoastlines = False,projection = dict(type = 'Mercator')))

fig = dict(data=data, layout=layout)
iplot(fig)

### It does not look like the number of days needed to process an application depends on the number of applications from that country - let's look at this data on a scatter plot and calculate some statistics ###

In [7]:
scl = map(str,(np.array(sns.cubehelix_palette(6, start=0, rot=1, dark=0.2, light=0.95))*255).astype(int))
scl = [w.replace('[ ','rgb(') for w in scl]
scl = [w.replace('[','rgb(') for w in scl]
scl= [w.replace(']',')') for w in scl]
scl= [w.replace('  ',' ') for w in scl]
scl= [w.replace(' ',',') for w in scl]
fr = np.linspace(0.0, 1.0, 6).tolist()

clrscl = [list(a) for a in zip(fr, scl)]

gbcountries['text'] = gbcountries['COUNTRY_OF_CITIZENSHIP'] + '<br>' +\
    'Median time '+ gbcountries['MEDIAN_TIME_TAKEN'].astype(int).astype(str) + '<br>' +\
     'Std '+ gbcountries['STD_TIME_TAKEN'].astype(int).astype(str) + '<br>' +\
      'No apps ' + gbcountries['count'].astype(str)

data = [ dict(type='choropleth',colorscale = clrscl,autocolorscale = False,locations = gbcountries['COUNTRY_OF_CITIZENSHIP_ABBR'],
        z = gbcountries['MEDIAN_TIME_TAKEN'],
        text = gbcountries['text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            )
        ),
        colorbar = dict(
            title = "Median time taken"
        )
    ) ]

layout = dict(title = 'Median time taken by Country<br>(Hover for breakdown)',geo = dict(showframe = False,showcoastlines = False,projection = dict(type = 'Mercator')))

fig = dict(data=data, layout=layout)
iplot(fig, filename='median-time-world-map')

### When looking at the median time instead of the mean, It looks like it takes around the same time to process applications from most countries. However, there are some exceptions - the countries seem to fall into 3 groups - where it takes < 150 days ; 150 - 300 days and > 300 days

In [8]:
err = ((gbcountries['STD_TIME_TAKEN'][gbcountries['count'] < 1000])/(gbcountries['count'][gbcountries['count'] < 1000]**(1/2.0)))*1.96



trace = Scatter(
    x = gbcountries['count'][gbcountries['count'] < 1000],
    y = gbcountries['MEAN_TIME_TAKEN'][gbcountries['count'] < 1000].astype(int),
    mode = 'markers',
    error_y=dict(
        type='data',
        array=err.astype(int),
        color='#85144B',
        thickness=0.5,
        width=2,
        opacity=0.4
        )
)

layout= Layout(title= 'No of days taken to process <1000 applications',
    xaxis= dict(
        title= 'Number of applications',
        zeroline = True,
    ),
    yaxis=dict(
        title= 'Number of days',
        zeroline = True,
    ),
    showlegend= False
)


fig= Figure(data=[trace], layout=layout)

iplot(fig, filename='scatter')

In [9]:
err = ((gbcountries['STD_TIME_TAKEN'][gbcountries['count'] >= 1000])/(gbcountries['count'][gbcountries['count'] >= 1000]**(1/2.0)))*1.96

trace = Scatter(
    x = gbcountries['count'][gbcountries['count'] >= 1000],
    y = gbcountries['MEAN_TIME_TAKEN'][gbcountries['count'] >= 1000].astype(int),
    mode = 'markers',
     error_y=dict(
        type='data',
        array=err.astype(int),
        color='#85144B',
        thickness=0.5,
        width=2,
        opacity=0.4
    )
    
)

layout= Layout(title= 'Mean and Standard error of number of days taken to process >1000 applications',
    xaxis= dict(
        title= 'Number of applications',
    ),
    yaxis=dict(
        title= 'Number of days',
    ),
    showlegend= False
)

fig= Figure(data=[trace], layout=layout)

iplot(fig, filename='scatter')

In [10]:
round(np.corrcoef(gbcountries['count'],  gbcountries['MEAN_TIME_TAKEN'])[0, 1].astype(float),4)

0.0185

###  The correlation coefficient is 0.0185 which shows that there is no correlation between time taken and number of applications. ###

In [11]:
gbstates = perm_data_15.groupby(['JOB_INFO_WORK_STATE']).size().reset_index(name='count')

gbstates['JOB_INFO_WORK_STATE_ABBR'] = ""

gbstates["MEAN_TIME_TAKEN"] = perm_data_15.groupby('JOB_INFO_WORK_STATE')['TIME_TAKEN'].mean().reset_index()['TIME_TAKEN']
gbstates["MEDIAN_TIME_TAKEN"] = perm_data_15.groupby('JOB_INFO_WORK_STATE')['TIME_TAKEN'].median().reset_index()['TIME_TAKEN']
gbstates["STD_TIME_TAKEN"] = perm_data_15.groupby('JOB_INFO_WORK_STATE')['TIME_TAKEN'].std().reset_index()['TIME_TAKEN']
gbstates['STD_TIME_TAKEN'] = gbstates['STD_TIME_TAKEN'].fillna(0)

# only keep 50 states to make plotting maps easier
todrop = [ 'FEDERATED STATES OF MICRONESIA', 'GUAM', 'MARSHALL ISLANDS','NORTHERN MARIANA ISLANDS', 'PUERTO RICO', 'VIRGIN ISLANDS']
for d in todrop:
    gbstates = gbstates[gbstates['JOB_INFO_WORK_STATE'] != d]
gbstates.index = range(len(gbstates))


# Create state abbreviations

for i, p in enumerate(gbstates['JOB_INFO_WORK_STATE']):
    if p == "UTAH":
        gbstates.set_value(i, "JOB_INFO_WORK_STATE", "UT")
    gbstates['JOB_INFO_WORK_STATE_ABBR'][i] = str(us.states.lookup(unicode(gbstates['JOB_INFO_WORK_STATE'][i])).abbr) 


/Users/sneharavi/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



### Which states get the most number of applications? ###

In [12]:
scl = map(str,(np.array(sns.cubehelix_palette(15, start=0, rot=0, dark=0.2, light=0.95))*255).astype(int))
scl = [w.replace('[ ','rgb(') for w in scl]
scl = [w.replace('[','rgb(') for w in scl]
scl= [w.replace(']',')') for w in scl]
scl= [w.replace('  ',' ') for w in scl]
scl= [w.replace(' ',',') for w in scl]
fr = np.linspace(0.0, 1.0, 15).tolist()

clrscl = [list(a) for a in zip(fr, scl)]
#print clrscl

gbstates['text'] = gbstates['JOB_INFO_WORK_STATE_ABBR'] + '<br>' +\
    'Number of applications '+ gbstates['count'].astype(str)

data = [ dict(type='choropleth',colorscale = clrscl,autocolorscale = False,locations = gbstates['JOB_INFO_WORK_STATE_ABBR'],
        z = gbstates['count'],
        locationmode = 'USA-states',
        text = gbstates['text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            )
        ),
        colorbar = dict(
            title = "Number of applications"
        )
    ) ]

layout = dict(title = 'Number of Applications by State<br>(Hover for breakdown)',geo = dict(scope='usa',projection=dict( type='albers usa' ),showlakes = True,lakecolor = 'rgb(255, 255, 255)'))
fig = dict(data=data, layout=layout)
iplot(fig, filename='apps-by-state-map')

In [13]:
countperc = (gbstates['count']/gbstates['count'].sum())*100
a =  countperc.sort_values(ascending = False)
b = np.array(countperc).argsort()[::-1][:len(a[a > 2.9])]

topcount = []
topperc = []
for val in a[b]:
    topperc.append(val)
topperc.append(int(100-sum(a[b])))
for c in gbstates.loc[b]['JOB_INFO_WORK_STATE_ABBR']:
        topcount.append(c)
topcount.append('OTHER STATES')

fig = {
    'data': [{'labels': topcount,
              'values': topperc,
              'type': 'pie',
            'hoverinfo':'none',
            'textinfo':'label+percent'}],
    'layout': {'title': 'Percentage of Applications by Prominent States'}
     }

iplot(fig)

### Is there a link between the number of days needed to process an application and the state  you apply to? ###

In [14]:
scl = map(str,(np.array(sns.cubehelix_palette(20, start=0, rot=-1, dark=0.2, light=0.95))*255).astype(int))
scl = [w.replace('[ ','rgb(') for w in scl]
scl = [w.replace('[','rgb(') for w in scl]
scl= [w.replace(']',')') for w in scl]
scl= [w.replace('  ',' ') for w in scl]
scl= [w.replace(' ',',') for w in scl]
fr = np.linspace(0.0, 1.0, 20).tolist()

clrscl = [list(a) for a in zip(fr, scl)]

gbstates['text'] = gbstates['JOB_INFO_WORK_STATE_ABBR'] + '<br>' +\
    'Mean time '+ gbstates['MEAN_TIME_TAKEN'].astype(int).astype(str) + '<br>' +\
     'Std '+ gbstates['STD_TIME_TAKEN'].astype(int).astype(str) + '<br>' +\
      'No apps ' + gbstates['count'].astype(str)

data = [ dict(type='choropleth',colorscale = clrscl,autocolorscale = False,locations = gbstates['JOB_INFO_WORK_STATE_ABBR'],
        z = gbstates['MEAN_TIME_TAKEN'],
        text = gbstates['text'],
        locationmode = 'USA-states',
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            )
        ),
        colorbar = dict(
            title = "Mean time taken"
        )
    ) ]

layout = dict(title = 'Mean time taken by State<br>(Hover for breakdown)',geo = dict(scope='usa',projection=dict( type='albers usa' ),showlakes = True,lakecolor = 'rgb(255, 255, 255)'))

fig = dict(data=data, layout=layout)
iplot(fig, filename='state-map')

#### Looks like there might be a link? Let's quantify this

In [15]:
scl = map(str,(np.array(sns.cubehelix_palette(20, start=0, rot=-1, dark=0.2, light=0.95))*255).astype(int))
scl = [w.replace('[ ','rgb(') for w in scl]
scl = [w.replace('[','rgb(') for w in scl]
scl= [w.replace(']',')') for w in scl]
scl= [w.replace('  ',' ') for w in scl]
scl= [w.replace(' ',',') for w in scl]
fr = np.linspace(0.0, 1.0, 20).tolist()

clrscl = [list(a) for a in zip(fr, scl)]

gbstates['text'] = gbstates['JOB_INFO_WORK_STATE_ABBR'] + '<br>' +\
    'Median time '+ gbstates['MEDIAN_TIME_TAKEN'].astype(int).astype(str) + '<br>' +\
     'Std '+ gbstates['STD_TIME_TAKEN'].astype(int).astype(str) + '<br>' +\
      'No apps ' + gbstates['count'].astype(str)

data = [ dict(type='choropleth',colorscale = clrscl,autocolorscale = False,locations = gbstates['JOB_INFO_WORK_STATE_ABBR'],
        z = gbstates['MEDIAN_TIME_TAKEN'],
        text = gbstates['text'],
        locationmode = 'USA-states',
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            )
        ),
        colorbar = dict(
            title = "Median time taken"
        )
    ) ]

layout = dict(title = 'Median time taken by State<br>(Hover for breakdown)',geo = dict(scope='usa',projection=dict( type='albers usa' ),showlakes = True,lakecolor = 'rgb(255, 255, 255)'))

fig = dict(data=data, layout=layout)
iplot(fig, filename='state-map')

#### It looks like median and mean are pretty similar and outliers are not affecting the data too much

In [16]:
err = ((gbstates['STD_TIME_TAKEN'])/(gbstates['count']**(1/2.0)))*1.96


trace = Scatter(
    x = gbstates['count'],
    y = gbstates['MEAN_TIME_TAKEN'].astype(int),
    mode = 'markers',
    error_y=dict(
        type='data',
        array=err.astype(int),
        color='#85144B',
        thickness=0.5,
        width=2,
        opacity=0.4
        )
)

layout= Layout(title= 'No of days taken to process applications by state',
    xaxis= dict(
        title= 'Number of applications',
        zeroline = True,
    ),
    yaxis=dict(
        title= 'Number of days',
        zeroline = True,
    ),
    showlegend= False
)


fig= Figure(data=[trace], layout=layout)
iplot(fig, filename='scatter state')

In [17]:
round(np.corrcoef(gbstates['count'],  gbstates['MEAN_TIME_TAKEN'])[0, 1].astype(float),4)

0.1913

###  The correlation coefficient is 0.1913 which shows that there is a very small correlation between time taken and number of applications processed by state. ###

### Does increased experience mean a faster processing time? ###

In [18]:
perm_data_15.iloc[0]

CASE_NUMBER                                        A-14220-96665
DECISION_DATE                                2015-02-03 00:00:00
CASE_STATUS                                    Certified-Expired
CASE_RECEIVED_DATE                           2014-09-10 00:00:00
EMPLOYER_NAME                             UNION PACIFIC RAILROAD
PW_JOB_TITLE_9089              Software Developers, Applications
PW_LEVEL_9089                                          Level III
WAGE_OFFER_FROM_9089                                   93,000.00
WAGE_OFFER_UNIT_OF_PAY_9089                                 Year
JOB_INFO_WORK_CITY                                         Omaha
JOB_INFO_WORK_STATE                                     NEBRASKA
COUNTRY_OF_CITIZENSHIP                                     INDIA
CLASS_OF_ADMISSION                                          H-1B
TIME_TAKEN                                                   146
Name: (15, 0), dtype: object

In [19]:
gbexper = perm_data_15.groupby(['PW_LEVEL_9089'])#.size().reset_index(name='count')
low = gbexper.get_group('Level I')['TIME_TAKEN'].reset_index()['TIME_TAKEN'].tolist() + gbexper.get_group('Level II')['TIME_TAKEN'].reset_index()['TIME_TAKEN'].tolist()
high = gbexper.get_group('Level III')['TIME_TAKEN'].reset_index()['TIME_TAKEN'].tolist() + gbexper.get_group('Level IV')['TIME_TAKEN'].reset_index()['TIME_TAKEN'].tolist()

trace0 = Box(y=low,name = 'Low Experience',boxmean=True, boxpoints=False)
trace1 = Box(y=high, name = 'High Experience',boxmean=True, boxpoints=False)
data = [trace0, trace1]
iplot(data)

In [20]:
import scipy as sp
sp.stats.ttest_ind(low, high, axis=0, equal_var=False)
#sp.stats.mannwhitneyu(low, high)


Ttest_indResult(statistic=9.9685838794384622, pvalue=2.1589942229176186e-23)

#### Higher experience does mean a faster processing time,  the 2 distributions are significantly different with a pvalue of 2e-23

In [21]:
#from plotly.tools import FigureFactory as FF

#fig = FF.create_violin(low, colors='#604d9e')
#iplot(fig, filename='One Violin')
#sns.violinplot(x="day", y="total_bill", data=low)


#np.array(low)
#np.array(high)
#error_y=dict(
#        type='data',
#        array=[(np.std(low)/np.sqrt(len(low)))*1.96],
#        visible=True
    
#       error_y=dict(
#        type='data',
#        array=[(np.std(high)/np.sqrt(len(high)))*1.96],
#        visible=True



#trace0 = Bar(
#    x=['Low Experience', 'High Experience'],
#    y=[np.mean(low), np.mean(high)],
#    marker=dict(color='rgb(158,202,225)',
#        line=dict(color='rgb(8,48,107)',width=2,)),
#    opacity=0.9
#)

#layout = Layout(
#    title='Level of experience and days to process data',
#    xaxis=dict(tickfont=dict(size=14,color='rgb(107, 107, 107)')),
#    yaxis=dict(title='No of days',titlefont=dict(size=16,color='rgb(107, 107, 107)'),tickfont=dict(size=14,color='rgb(107, 107, 107)')),
#)

#fig =Figure(data=[trace0], layout=layout)
#iplot(fig, filename='bar-plot')

In [22]:
gbadm = perm_data_15.groupby(['CLASS_OF_ADMISSION']).size().reset_index(name='count')

gbadm["MEAN_TIME_TAKEN"] = perm_data_15.groupby('CLASS_OF_ADMISSION')['TIME_TAKEN'].mean().reset_index()['TIME_TAKEN'].astype(int)
#gbadm["MEDIAN_TIME_TAKEN"] = perm_data_15.groupby('CLASS_OF_ADMISSION')['TIME_TAKEN'].median().reset_index()['TIME_TAKEN'].astype(int)
gbadm["STD_TIME_TAKEN"] = perm_data_15.groupby('CLASS_OF_ADMISSION')['TIME_TAKEN'].std().reset_index()['TIME_TAKEN']
gbadm['STD_TIME_TAKEN'] = gbadm['STD_TIME_TAKEN'].fillna(0)
gbadm['STD_TIME_TAKEN'] = gbadm['STD_TIME_TAKEN'].astype(int)



In [23]:
gbjob = perm_data_15.groupby(['PW_JOB_TITLE_9089']).size().reset_index(name='count')

gbjob["MEAN_TIME_TAKEN"] = perm_data_15.groupby('PW_JOB_TITLE_9089')['TIME_TAKEN'].mean().reset_index()['TIME_TAKEN'].astype(int)
#gbadm["MEDIAN_TIME_TAKEN"] = perm_data_15.groupby('CLASS_OF_ADMISSION')['TIME_TAKEN'].median().reset_index()['TIME_TAKEN'].astype(int)
gbjob["STD_TIME_TAKEN"] = perm_data_15.groupby('PW_JOB_TITLE_9089')['TIME_TAKEN'].std().reset_index()['TIME_TAKEN']
gbjob['STD_TIME_TAKEN'] = gbjob['STD_TIME_TAKEN'].fillna(0)


In [24]:
gbmonth = perm_data_15.groupby(pd.DatetimeIndex(perm_data_15['CASE_RECEIVED_DATE']).month).size().reset_index(name='count')
gbmonth["MEAN_TIME_TAKEN"] = perm_data_15.groupby(pd.DatetimeIndex(perm_data_15['CASE_RECEIVED_DATE']).month)['TIME_TAKEN'].mean().reset_index()['TIME_TAKEN'].astype(int)
#gbadm["MEDIAN_TIME_TAKEN"] = perm_data_15.groupby('CLASS_OF_ADMISSION')['TIME_TAKEN'].median().reset_index()['TIME_TAKEN'].astype(int)
gbmonth["STD_TIME_TAKEN"] = perm_data_15.groupby(pd.DatetimeIndex(perm_data_15['CASE_RECEIVED_DATE']).month)['TIME_TAKEN'].std().reset_index()['TIME_TAKEN']
gbmonth['STD_TIME_TAKEN'] = gbmonth['STD_TIME_TAKEN'].fillna(0)
#gbmonth


### Machine Learning

#### Let's see if I can predict how long it will take someone to get a permanent visa based on some of the features visualized above - where they are from, where they are applying to, level of experience and the visa class they are currently on: 

#### Let's try using linear regression to predict the number of days it will take with these features - all 50 states, 137 countries, 4 levels of experience and 48 visa classes . I use one hot encoding since these are all categorical features.


In [25]:
from sklearn.preprocessing import OneHotEncoder

perm_data_trunc = perm_data_15#[perm_data_15["TIME_TAKEN"] < 800]

feature_cols = ['COUNTRY_OF_CITIZENSHIP','PW_LEVEL_9089', 'CLASS_OF_ADMISSION', 'JOB_INFO_WORK_STATE'] 
x = perm_data_trunc[feature_cols].reset_index()[feature_cols]
y = perm_data_trunc['TIME_TAKEN'].reset_index()['TIME_TAKEN']

perm_dict = {}
perm_dict['COUNTRY_OF_CITIZENSHIP'] = dict(zip( np.unique(x['COUNTRY_OF_CITIZENSHIP']) ,range(len(np.unique(x['COUNTRY_OF_CITIZENSHIP'])))))
perm_dict['PW_LEVEL_9089'] = dict(zip( np.unique(x['PW_LEVEL_9089']) ,range(len(np.unique(x['PW_LEVEL_9089'])))))
perm_dict['CLASS_OF_ADMISSION'] = dict(zip( np.unique(x['CLASS_OF_ADMISSION']) ,range(len(np.unique(x['CLASS_OF_ADMISSION'])))))
perm_dict['JOB_INFO_WORK_STATE'] = dict(zip( np.unique(x['JOB_INFO_WORK_STATE']) ,range(len(np.unique(x['JOB_INFO_WORK_STATE'])))))

x = x.replace(to_replace = perm_dict)
enc = OneHotEncoder()
x2= enc.fit_transform(x)
xnew = x2.todense()

from sklearn import datasets, metrics
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import train_test_split
lm = LinearRegression()
x_train,x_test,y_train,y_test = train_test_split(xnew, y,test_size=0.2)

lm.fit(x_train, y_train)

# The coefficients
#print('Coefficients: \n', lm.coef_)
# The mean square error
print("Sum of squares error: %.2f"
      % np.mean((lm.predict(x_test) - y_test) ** 2))
# Explained variance score: 1 is perfect prediction
print('R squared value: %.2f' % lm.score(x_test, y_test))

Sum of squares error: 5309806563008810026270720.00
R squared value: -131008029353826516992.00


#### The SSE is very high and the R2 value is highly negative showing that the prediction by this model is quite poor. 
#### It could be that I am using too many features to train my model and predict the responses 
#### 50 states, 137 countries, 4 levels of experience and 48 visa classes
#### I am next going to try reducing the number of features by grouping each feature into 4 categories - countries, states and visa classes where the processing time was < 200 days; 200-300; 300-400 or > 400 days. I picked these 4 values based on the distribution I observed from the histogram below.

In [26]:
data = [
    Histogram(
        x=gbcountries["MEAN_TIME_TAKEN"]
    )
]

layout= Layout(title= 'Mean time taken to process visa across different countries',
    xaxis= dict(
        title= 'Mean number of days',
        zeroline = True,
    ),
    yaxis=dict(
        title= 'Frequency',
        zeroline = True,
    ),
    showlegend= False
)


fig =Figure(data=data, layout=layout)
iplot(fig, filename='countries-histogram')

In [27]:
gbcountriesall = perm_data_15.groupby(['COUNTRY_OF_CITIZENSHIP']).size().reset_index(name='count')
gbcountriesall["MEAN_TIME_TAKEN"] = perm_data_15.groupby('COUNTRY_OF_CITIZENSHIP')['TIME_TAKEN'].mean().reset_index()['TIME_TAKEN']

coun100 = gbcountriesall[gbcountriesall["MEAN_TIME_TAKEN"] < 200]["COUNTRY_OF_CITIZENSHIP"].tolist()
a = gbcountriesall[gbcountriesall["MEAN_TIME_TAKEN"] < 300]
coun200  = a[a["MEAN_TIME_TAKEN"] > 200]["COUNTRY_OF_CITIZENSHIP"].tolist()
b = gbcountriesall[gbcountriesall["MEAN_TIME_TAKEN"] < 400]
coun300  = b[b["MEAN_TIME_TAKEN"] > 300]["COUNTRY_OF_CITIZENSHIP"].tolist()
coun400 = gbcountriesall[gbcountriesall["MEAN_TIME_TAKEN"] > 400]["COUNTRY_OF_CITIZENSHIP"].tolist()


def countriesclass(x):
    if x in coun100:
        return '1'
    elif x in coun200:
        return '2'
    elif x in coun300:
        return '3'
    elif x in coun400:
        return '4'
    
gbadm = perm_data_15.groupby(['CLASS_OF_ADMISSION']).size().reset_index(name='count')
gbadm["MEAN_TIME_TAKEN"] = perm_data_15.groupby('CLASS_OF_ADMISSION')['TIME_TAKEN'].mean().reset_index()['TIME_TAKEN'].astype(int)

adm100 = gbadm[gbadm["MEAN_TIME_TAKEN"] < 200]["CLASS_OF_ADMISSION"].tolist()
a = gbadm[gbadm["MEAN_TIME_TAKEN"] < 300]
adm200  = a[a["MEAN_TIME_TAKEN"] > 200]["CLASS_OF_ADMISSION"].tolist()
b = gbadm[gbadm["MEAN_TIME_TAKEN"] < 400]
adm300  = b[b["MEAN_TIME_TAKEN"] > 300]["CLASS_OF_ADMISSION"].tolist()
adm400 = gbadm[gbadm["MEAN_TIME_TAKEN"] > 400]["CLASS_OF_ADMISSION"].tolist()


def admclass(x):
    if x in adm100:
        return '1'
    elif x in adm200:
        return '2'
    elif x in adm300:
        return '3'
    elif x in adm400:
        return '4'

gbstatesall = perm_data_15.groupby(['JOB_INFO_WORK_STATE']).size().reset_index(name='count')
gbstatesall["MEAN_TIME_TAKEN"] = perm_data_15.groupby('JOB_INFO_WORK_STATE')['TIME_TAKEN'].mean().reset_index()['TIME_TAKEN'].astype(int)

st100 = gbstatesall[gbstatesall["MEAN_TIME_TAKEN"] < 200]["JOB_INFO_WORK_STATE"].tolist()
a = gbstatesall[gbstatesall["MEAN_TIME_TAKEN"] < 300]
st200  = a[a["MEAN_TIME_TAKEN"] > 200]["JOB_INFO_WORK_STATE"].tolist()
b = gbstatesall[gbstatesall["MEAN_TIME_TAKEN"] < 400]
st300  = b[b["MEAN_TIME_TAKEN"] > 300]["JOB_INFO_WORK_STATE"].tolist()
st400 = gbstatesall[gbstatesall["MEAN_TIME_TAKEN"] > 400]["JOB_INFO_WORK_STATE"].tolist()


def stclass(x):
    if x in st100:
        return '1'
    elif x in st200:
        return '2'
    elif x in st300:
        return '3'
    elif x in st400:
        return '4'
    
perm_data_15['ADMISSION_CLASS'] = perm_data_15['CLASS_OF_ADMISSION'].apply(admclass)
perm_data_15['COUNTRIES_CLASS'] = perm_data_15['COUNTRY_OF_CITIZENSHIP'].apply(countriesclass)
perm_data_15['STATES_CLASS'] = perm_data_15['JOB_INFO_WORK_STATE'].apply(stclass)

### Running Linear Regression again after reducing the number of features ###

In [28]:
perm_data_trunc = perm_data_15#[perm_data_15["TIME_TAKEN"] < 800]

feature_cols = ['COUNTRIES_CLASS','PW_LEVEL_9089', 'ADMISSION_CLASS', 'STATES_CLASS'] 
x = perm_data_trunc[feature_cols].reset_index()[feature_cols]
y = perm_data_trunc['TIME_TAKEN'].reset_index()['TIME_TAKEN']

perm_dict = {}
perm_dict['PW_LEVEL_9089'] = dict(zip( np.unique(x['PW_LEVEL_9089']) ,range(len(np.unique(x['PW_LEVEL_9089'])))))
perm_dict['COUNTRIES_CLASS'] = dict(zip( np.unique(x['COUNTRIES_CLASS']) ,range(len(np.unique(x['COUNTRIES_CLASS'])))))
perm_dict['ADMISSION_CLASS'] = dict(zip( np.unique(x['ADMISSION_CLASS']) ,range(len(np.unique(x['ADMISSION_CLASS'])))))
perm_dict['STATES_CLASS'] = dict(zip( np.unique(x['STATES_CLASS']) ,range(len(np.unique(x['STATES_CLASS'])))))

x = x.replace(to_replace = perm_dict)
enc = OneHotEncoder()
x2= enc.fit_transform(x)
xnew = x2.todense()

x_train,x_test,y_train,y_test = train_test_split(xnew, y,test_size=0.2)

lm.fit(x_train, y_train)

# The coefficients
#print('Coefficients: \n', lm.coef_)
# The mean square error
print("Sum of Sqaures Error: %.2f"
      % np.mean((lm.predict(x_test) - y_test) ** 2))
# Explained variance score: 1 is perfect prediction
print('R squared value: %.2f' % lm.score(x_test, y_test))
print('Mean of data: %.2f' % np.mean(y_test))
print('Standard deviation in data: %.2f' % np.std(y_test))
print('Mean of prediction: %.2f' % np.mean(lm.predict(x_test)))
print('Standard deviation in prediction: %.2f' % np.std(lm.predict(x_test)))



Sum of Sqaures Error: 43282.19
R squared value: 0.03
Mean of data: 283.09
Standard deviation in data: 211.51
Mean of prediction: 283.76
Standard deviation in prediction: 40.15


#### Reducing the number of features significantly decreases the sum of squares error and improves the R squared. 
#### However, the R squared value is only 0.04, which means that there is still room for improvement in the linear model. Few things that can be done:

#### 1) Define new features which are better predictors 
#### 2) Try Polynomial Regression
#### 3) Try normalizing the y value - number of days
#### 4) Reguarization
#### 5) Get more data
#### 6) Try Logistic instead of Linear Regression

### When looking at why Linear Regression did not give good performance - I noticed that part of the reason was because there is a lot of variance in the data. The number of days needed can vary by a few months, and so an exact prediction will not be very precise and the SSE will be very large. Thus, instead of trying to predict the exact number, I am going to try to predict whether the visas wil be processed within 4 months, within 4-8 months ; 8 - 12 months; 12-16 months; 16-20 months or > 20 months using Logistic Regression. This will still be very useful to someone to check what their processing time could be based on features in their application.

In [29]:
#data = [
#    Histogram(
#        x= abs(lm.predict(x_test) - y_test)
#    )
#]
#iplot(data)
lm.predict(x_test) - y_test

6493      128.195312
75892     125.828125
32556     156.890625
27676    -272.578125
5089       83.640625
34530    -281.804688
13930    -308.804688
33317      50.007812
16753     129.195312
29587     120.828125
38648     -34.359375
8651       79.195312
24092    -372.742188
69944     114.828125
63059      42.234375
79304     112.195312
39686     112.257812
69678      87.640625
50665      58.234375
28823     -86.140625
27639    -333.992188
66386     105.640625
77924      81.007812
70789      92.890625
17596    -183.992188
55069   -1644.171875
20927    -247.804688
35659    -130.718750
29479      55.007812
41009    -285.359375
94       -480.171875
69030      94.867188
72699      72.234375
14542     173.828125
25133    -173.742188
47512      87.234375
19494      52.007812
20891    -268.937500
36252     110.007812
77039      95.195312
69167      67.007812
74741     106.062500
65085      55.195312
41029    -608.742188
64081      38.234375
17312    -284.765625
46663      44.234375
64859      65

In [30]:
# Define categories:

perm_data_15.ix[perm_data_15["TIME_TAKEN"] >= 600, 'TIME_CATEGORIES'] = 6
perm_data_15.ix[perm_data_15["TIME_TAKEN"] < 600, 'TIME_CATEGORIES'] = 5
perm_data_15.ix[perm_data_15["TIME_TAKEN"] < 480, 'TIME_CATEGORIES'] = 4
perm_data_15.ix[perm_data_15["TIME_TAKEN"] < 360, 'TIME_CATEGORIES'] = 3
perm_data_15.ix[perm_data_15["TIME_TAKEN"] < 240, 'TIME_CATEGORIES'] = 2
perm_data_15.ix[perm_data_15["TIME_TAKEN"] < 120, 'TIME_CATEGORIES'] = 1

In [32]:
perm_data_trunc = perm_data_15#[perm_data_15["TIME_TAKEN"] < 600]

feature_cols = ['COUNTRIES_CLASS','PW_LEVEL_9089', 'ADMISSION_CLASS', 'STATES_CLASS'] 
x = perm_data_trunc[feature_cols].reset_index()[feature_cols]
y = perm_data_trunc['TIME_CATEGORIES'].reset_index()['TIME_CATEGORIES'].tolist()

perm_dict = {}
perm_dict['COUNTRIES_CLASS'] = dict(zip( np.unique(x['COUNTRIES_CLASS']) ,range(len(np.unique(x['COUNTRIES_CLASS'])))))
perm_dict['ADMISSION_CLASS'] = dict(zip( np.unique(x['ADMISSION_CLASS']) ,range(len(np.unique(x['ADMISSION_CLASS'])))))
perm_dict['STATES_CLASS'] = dict(zip( np.unique(x['STATES_CLASS']) ,range(len(np.unique(x['STATES_CLASS'])))))
perm_dict['PW_LEVEL_9089'] = dict(zip( np.unique(x['PW_LEVEL_9089']) ,range(len(np.unique(x['PW_LEVEL_9089'])))))

from sklearn import linear_model, datasets, metrics
#from sklearn.cross_validation import train_test_split
x = x.replace(to_replace = perm_dict)
enc = OneHotEncoder()
x2= enc.fit_transform(x)
xnew = x2.todense()

logreg = linear_model.LogisticRegression()
X_train,X_test,y_train,y_test = train_test_split(xnew,y,test_size=0.2)
L = logreg.fit(X_train,y_train)
y_pred = logreg.predict(X_test)

print 'Accuracy of model on test set is ' + str(metrics.accuracy_score(y_test,y_pred)*100) +' %'

Accuracy of model on test set is 70.4367506729 %


In [33]:
len(perm_data_15[perm_data_15["TIME_TAKEN"] < 600])/float(len(perm_data_15))

0.9557132370932224

#### The Logistic Regression model does much better at predicting processing times within 4 month windows. #### 
#### The histogram below reflects the fact that 96% of the points fall within ~600 days - let's try eliminating these outliers and see if we  get a better prediction

In [34]:
data = [
    Histogram(
        x=perm_data_15["TIME_TAKEN"]
    )
]

layout= Layout(title= 'Distribution of time taken to process visa',
    xaxis= dict(
        title= 'Number of days',
        zeroline = True,
    ),
    yaxis=dict(
        title= 'Frequency',
        zeroline = True,
    ),
    showlegend= False
)

fig =Figure(data=data, layout=layout)
iplot(fig, filename='days-histogram')

In [35]:
perm_data_trunc = perm_data_15[perm_data_15["TIME_TAKEN"] < 600]

feature_cols = ['COUNTRIES_CLASS','PW_LEVEL_9089', 'ADMISSION_CLASS', 'STATES_CLASS'] 
x = perm_data_trunc[feature_cols].reset_index()[feature_cols]
y = perm_data_trunc['TIME_CATEGORIES'].reset_index()['TIME_CATEGORIES'].tolist()

perm_dict = {}
perm_dict['COUNTRIES_CLASS'] = dict(zip( np.unique(x['COUNTRIES_CLASS']) ,range(len(np.unique(x['COUNTRIES_CLASS'])))))
perm_dict['ADMISSION_CLASS'] = dict(zip( np.unique(x['ADMISSION_CLASS']) ,range(len(np.unique(x['ADMISSION_CLASS'])))))
perm_dict['STATES_CLASS'] = dict(zip( np.unique(x['STATES_CLASS']) ,range(len(np.unique(x['STATES_CLASS'])))))
perm_dict['PW_LEVEL_9089'] = dict(zip( np.unique(x['PW_LEVEL_9089']) ,range(len(np.unique(x['PW_LEVEL_9089'])))))

x = x.replace(to_replace = perm_dict)
enc = OneHotEncoder()
x2= enc.fit_transform(x)
xnew = x2.todense()

logreg = linear_model.LogisticRegression()
X_train,X_test,y_train,y_test = train_test_split(xnew,y,test_size=0.2)
L = logreg.fit(X_train,y_train)
y_pred = logreg.predict(X_test)

print 'Accuracy of model on test set is ' + str(metrics.accuracy_score(y_test,y_pred)*100) +' %'

Accuracy of model on test set is 73.9311315924 %


#### The accuracy improves! Let's see how well the model does with true and false positives

In [36]:
print metrics.confusion_matrix(y_test, y_pred)
print ("Predicting everything as falling within 4-8 months will still give accuracy of "+ str((len([x for x in y_test if x == 2])/float(len(y_test)))*100))

[[    0    93     0     6     0]
 [    0 11518     0    13     4]
 [    0   633     0     3     2]
 [    0  1539     0    23    13]
 [    0  1743     0    24    10]]
Predicting everything as falling within 4-8 months will still give accuracy of 73.8287250384


#### However, if we look carefully at the confusion matrix, we see that most times are predicted as falling within 4-8 months.
#### The logistic regression doesn't perform much better than predicting everything as falling within 4-8 months. This is probably because the dataset is quite skewed towards these values.

#### When looking at the data closely, it seems like most of the applications that are processed within 120 days were denied. Adding that feature might improve the prediction by the model - though it is not something users will know when they key in details about their application 

In [37]:
perm_data_15[perm_data_15['TIME_CATEGORIES']==1].head()

CASE_NUMBER DECISION_DATE CASE_STATUS CASE_RECEIVED_DATE  \
15 79034  A-15134-76322    2015-07-07      Denied         2015-06-01   
   79044  A-15138-77412    2015-06-23      Denied         2015-05-18   
   79059  A-14358-37833    2015-02-18      Denied         2015-01-13   
   79119  A-14304-21633    2015-01-09      Denied         2014-11-01   
   79121  A-15180-92423    2015-07-29      Denied         2015-07-29   

                         EMPLOYER_NAME                  PW_JOB_TITLE_9089  \
15 79034  E-BUSINESS INTERNATIONAL INC   SOFTWARE DEVELOPER, APPLICATIONS   
   79044   HEXAWARE TECHNOLOGIES, INC.  Software Developers, Applications   
   79059                   INOVANT LLC  Software Developers, Applications   
   79119              GCB SERVICES LLC                 SENIOR RF ENGINEER   
   79121                FACEBOOK, INC.  Software Developers, Applications   

         PW_LEVEL_9089 WAGE_OFFER_FROM_9089 WAGE_OFFER_UNIT_OF_PAY_9089  \
15 79034      Level II            77,210.00                        Year   
   79044      Level IV           108,805.00                        Year   
   79059     Level III           110,000.00                        Year   
   79119      Level IV           123,000.00                        Year   
   79121      Level II           147,977.86                        Year   

         JOB_INFO_WORK_CITY JOB_INFO_WORK_STATE COUNTRY_OF_CITIZENSHIP  \
15 79034     EAST BRUNSWICK          NEW JERSEY                  INDIA   
   79044          Jamesburg          NEW JERSEY                  INDIA   
   79059             Austin               TEXAS                  INDIA   
   79119             MCLEAN            VIRGINIA                  INDIA   
   79121         Menlo Park          CALIFORNIA             LUXEMBOURG   

         CLASS_OF_ADMISSION  TIME_TAKEN ADMISSION_CLASS COUNTRIES_CLASS  \
15 79034               H-1B          36               2               2   
   79044               H-1B          36               2               2   
   79059               H-1B          36               2               2   
   79119              H-1B1          69               2               2   
   79121               H-1B           0               2               1   

         STATES_CLASS  TIME_CATEGORIES  
15 79034            3              1.0  
   79044            3              1.0  
   79059            3              1.0  
   79119            2              1.0  
   79121            2              1.0

In [38]:
perm_data_15.ix[perm_data_15['CASE_STATUS'].str.startswith("C"), 'ACCEPTED_CLASS'] = '2'
perm_data_15.ix[perm_data_15['CASE_STATUS'].str.startswith("D"), 'ACCEPTED_CLASS'] = '1'

In [39]:
perm_data_trunc = perm_data_15[perm_data_15["TIME_TAKEN"] < 600]

feature_cols = ['COUNTRIES_CLASS','PW_LEVEL_9089', 'ADMISSION_CLASS', 'STATES_CLASS', 'ACCEPTED_CLASS'] 
x = perm_data_trunc[feature_cols].reset_index()[feature_cols]
y = perm_data_trunc['TIME_CATEGORIES'].reset_index()['TIME_CATEGORIES'].tolist()

perm_dict = {}
perm_dict['COUNTRIES_CLASS'] = dict(zip( np.unique(x['COUNTRIES_CLASS']) ,range(len(np.unique(x['COUNTRIES_CLASS'])))))
perm_dict['ADMISSION_CLASS'] = dict(zip( np.unique(x['ADMISSION_CLASS']) ,range(len(np.unique(x['ADMISSION_CLASS'])))))
perm_dict['STATES_CLASS'] = dict(zip( np.unique(x['STATES_CLASS']) ,range(len(np.unique(x['STATES_CLASS'])))))
perm_dict['PW_LEVEL_9089'] = dict(zip( np.unique(x['PW_LEVEL_9089']) ,range(len(np.unique(x['PW_LEVEL_9089'])))))
perm_dict['ACCEPTED_CLASS'] = dict(zip( np.unique(x['ACCEPTED_CLASS']) ,range(len(np.unique(x['ACCEPTED_CLASS'])))))

x = x.replace(to_replace = perm_dict)
enc = OneHotEncoder()
x2= enc.fit_transform(x)
xnew = x2.todense()

logreg = linear_model.LogisticRegression()
X_train,X_test,y_train,y_test = train_test_split(xnew,y,test_size=0.2)
L = logreg.fit(X_train,y_train)
y_pred = logreg.predict(X_test)

print 'Accuracy of model on test set is ' + str(metrics.accuracy_score(y_test,y_pred)*100) +' %'

Accuracy of model on test set is 73.9503328213 %


In [40]:
print metrics.confusion_matrix(y_test, y_pred)
print ("Predicting everything as falling within 4-8 months will still give accuracy of "+ str((len([x for x in y_test if x == 2])/float(len(y_test)))*100))

[[    0    15     7    37    14]
 [    0 11384     9    45    41]
 [    0   612     9    14    26]
 [    1  1432    14   112    37]
 [    0  1655    20    91    49]]
Predicting everything as falling within 4-8 months will still give accuracy of 73.4703020993


#### Model does a little bit better, but there is still romm for improvement ####

#### One can try: Including more features like wage amount, adding more data, regularization and grid search, trying random forest to pick good features

#### Next step: Build a web app, Try SQL, Try scraping web images

In [ ]:
#gbmonth = perm_data_15.groupby(pd.DatetimeIndex(perm_data_15['CASE_RECEIVED_DATE']).month).size().reset_index(name='count')
#gbmonth["MEAN_TIME_TAKEN"] = perm_data_15.groupby(pd.DatetimeIndex(perm_data_15['CASE_RECEIVED_DATE']).month)['TIME_TAKEN'].mean().reset_index()['TIME_TAKEN'].astype(int)

#mth200 = gbmonth[gbmonth["MEAN_TIME_TAKEN"] < 250]['index'].tolist()
#a = gbmonth[gbmonth["MEAN_TIME_TAKEN"] < 300]
#mth250  = a[a["MEAN_TIME_TAKEN"] > 250]["index"].tolist()
#b = gbmonth[gbmonth["MEAN_TIME_TAKEN"] < 350]
#mth300  = b[b["MEAN_TIME_TAKEN"] > 300]["index"].tolist()
#mth350 = gbmonth[gbmonth["MEAN_TIME_TAKEN"] > 350]["index"].tolist()


#def mthclass(x):
#    if x in mth200:
#        return '1'
#    elif x in mth250:
#        return '2'
#    elif x in mth300:
#        return '3'
#    elif x in mth350:
#        return '4'   


#perm_data_15['MONTH_CLASS'] = pd.Series(pd.DatetimeIndex(perm_data_15['CASE_RECEIVED_DATE']).month).apply(mthclass)

#month, logistic regression 4 classes, accepted vs denied, reduce extreme points

In [ ]:
#np.mean(lm.predict(x_test))
#np.std(lm.predict(x_test))
#np.mean(y_test)
#np.std(y_test)
#lm.predict(x_test) - y_test


In [ ]:
#from sklearn.preprocessing import OneHotEncoder

#perm_data_trunc = perm_data_15#[perm_data_15["TIME_TAKEN"] < 600]

#feature_cols = ['JOB_INFO_WORK_STATE', 'COUNTRY_OF_CITIZENSHIP', 'CLASS_OF_ADMISSION', 'PW_LEVEL_9089'] 
#x = perm_data_trunc[feature_cols].reset_index()[feature_cols]
#y = perm_data_trunc['TIME_CATEGORIES'].reset_index()['TIME_CATEGORIES'].tolist()

#perm_dict = {}
#perm_dict['COUNTRY_OF_CITIZENSHIP'] = dict(zip( np.unique(x['COUNTRY_OF_CITIZENSHIP']) ,range(len(np.unique(x['COUNTRY_OF_CITIZENSHIP'])))))
#perm_dict['PW_LEVEL_9089'] = dict(zip( np.unique(x['PW_LEVEL_9089']) ,range(len(np.unique(x['PW_LEVEL_9089'])))))
#perm_dict['CLASS_OF_ADMISSION'] = dict(zip( np.unique(x['CLASS_OF_ADMISSION']) ,range(len(np.unique(x['CLASS_OF_ADMISSION'])))))
#perm_dict['JOB_INFO_WORK_STATE'] = dict(zip( np.unique(x['JOB_INFO_WORK_STATE']) ,range(len(np.unique(x['JOB_INFO_WORK_STATE'])))))

#x = x.replace(to_replace = perm_dict)
#enc = OneHotEncoder()
#x2= enc.fit_transform(x)
#xnew = x2.todense()

#from sklearn import linear_model, datasets, metrics
#from sklearn.cross_validation import train_test_split
#logreg = linear_model.LogisticRegression()
#X_train,X_test,y_train,y_test = train_test_split(xnew,y,test_size=0.2)
#L = logreg.fit(X_train,y_train)
#y_pred = logreg.predict(X_test)


#print 'Accuracy of model on test set is ' + str(metrics.accuracy_score(y_test,y_pred)*100) +' %'



In [ ]:
#for cvals in [0.01]:
#    logreg = linear_model.LogisticRegression(class_weight='balanced',C=cvals)
#    X_train,X_test,y_train,y_test = train_test_split(xnew,y,test_size=0.2)
#    L = logreg.fit(X_train,y_train)
#    y_pred = logreg.predict(X_test)
#    
#    print "# C = " +str(cvals)
#    
#    print 'Null Accuracy is ' + str(max(np.mean(y_test),1-np.mean(y_test))*100) + '%'

#    confusion = metrics.confusion_matrix(y_test, y_pred)
#    print confusion
#    TP = confusion[1, 1]
#    TN = confusion[0, 0]
#    FP = confusion[0, 1]
#    FN = confusion[1, 0]
#    print "Sensitivity = Recall = %f" % metrics.recall_score(y_test, y_pred)
#    print "Specificity = %f" % (TN / float(TN + FP))
#    print "Precision = %f" % metrics.precision_score(y_test, y_pred)

In [ ]:
#y_pred_prob = logreg.predict_proba(X_test)[:, 1]

#fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_prob)
#plt.plot(fpr, tpr)
#plt.xlim([0.0, 1.0])
#plt.ylim([0.0, 1.0])
##plt.title('ROC curve for Prostitution classifier')
#plt.xlabel('False Positive Rate (1 - Specificity)')
#plt.ylabel('True Positive Rate (Sensitivity)')
#plt.grid(True)

In [ ]:
#trace = Scatter(
#    x = perm_data_15['WAGE_OFFER_FROM_9089'],
#    y = perm_data_15['TIME_TAKEN'].astype(int),
#    mode = 'markers',
#)

#layout= Layout(title= 'No of days taken to process applications by state',
#    xaxis= dict(
#        title= 'Number of applications',
#        zeroline = True,
#    ),
#    yaxis=dict(
#        title= 'Number of days',
 #       zeroline = True,
#    ),
#    showlegend= False
#)


#fig= Figure(data=[trace], layout=layout)
#iplot(fig, filename='scatter state')

In [ ]:
#trace = Scatter(
#    x = perm_data_15[perm_data_15['WAGE_OFFER_UNIT_OF_PAY_9089'] == 'Year']['WAGE_OFFER_FROM_9089'],
#    y = perm_data_15[perm_data_15['WAGE_OFFER_UNIT_OF_PAY_9089'] == 'Year']['TIME_TAKEN'],
#    mode = 'markers',
#)
#fig= Figure(data=[trace], layout=layout)
#iplot(fig, filename='scatter state')

In [ ]:
#a[a == '#############']
#a = perm_data_15[perm_data_15['WAGE_OFFER_UNIT_OF_PAY_9089'] == 'Year']['WAGE_OFFER_FROM_9089']
#int(a[0].replace(',', ''))
#b = a[0]
#map(float,a)
#a = perm_data_15[perm_data_15['WAGE_OFFER_UNIT_OF_PAY_9089'] == 'Year']['WAGE_OFFER_FROM_9089'].astype(str)
#a = [a.replace(',', '') for a in a]
#a = [a.replace(' ', '0') for a in a]

#pd.Dataframe(a).astype(int)
#a